# JulES as a medium-term prognosis model

### Import packages

In [14]:
#Pkg.instantiate()
using Pkg; Pkg.status()
#Pkg.add("CSV"); Pkg.add("Revise"); Pkg.add("Plots"); Pkg.add("PlotlyJS"); Pkg.add("PrettyTables")
# Pkg.update("TuLiPa") # uncomment to update TuLiPa to latest version
# Pkg.develop(path=joinpath(dirname(dirname(pwd())),"TuLiPa")); Pkg.status() # go to development version
#Pkg.undo(); Pkg.status() # go back to main package version
#Pkg.add(url="https://github.com/NVE/TuLiPa.git", rev="redesign_JulES"); Pkg.status() # alternative go back to latest version

Project JulES v0.1.0
Status `X:\Prosjekter\2022_FoU_markedsmodell_julia\Brukere\mary\JulES\Project.toml`
  [336ed68f] CSV v0.10.14
  [aaaa29a8] Clustering v0.15.7
  [a93c6f00] DataFrames v1.6.1
  [aaf54ef3] DistributedArrays v0.6.7
⌃ [31c24e10] Distributions v0.25.107
  [7073ff75] IJulia v1.24.2
  [682c06a0] JSON v0.21.4
  [f0f68f2c] PlotlyJS v0.18.13
  [91a5bcdd] Plots v1.40.4
⌃ [aea7be01] PrecompileTools v1.2.0
  [08abe8d2] PrettyTables v2.3.2
  [295af30f] Revise v3.5.14
  [970f5c25] TuLiPa v0.1.0 `https://github.com/NVE/TuLiPa.git#redesign_JulES`
⌃ [ddb6d928] YAML v0.4.9
  [ade2ca70] Dates
  [8ba89e20] Distributed
  [10745b16] Statistics v1.9.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [15]:
using DataFrames, Statistics, JSON, Distributed, Clustering, YAML, Distributions, Revise, Plots, PrettyTables, Random
plotlyjs(); # uncomment for interactive plots

In [16]:
# config = YAML.load_file(joinpath("data", "config_jules_prognose.yml")) # config without datasets
config = YAML.load_file(joinpath("data", "config_jules_prognose_demo.yml")) # config with NVE datasets
weatheryear = config["main"]["weatheryears"][1]
datayear = config["main"]["datayears"][1]

2024

### Prepare parallell processing - import code on all cores

In [17]:
const numcores = config["main"]["numcores"]

if nprocs() < numcores
    addprocs(numcores - nprocs())
end

@show nprocs();

nprocs() = 10


In [18]:
@everywhere using TuLiPa, Dates
# @everywhere include(joinpath(dirname(dirname(pwd())),"TuLiPa/src/TuLiPa.jl"));

In [19]:
@everywhere using JulES

In [20]:
function getdataset(config, weatheryear)
    settings = config[config["main"]["settings"]]

    sti_dataset = joinpath(config["main"]["inputpath"], "static_input")
    weekstart = config["main"]["weekstart"]

    sti_dataset1 = joinpath(config["main"]["inputpath"], "Uke_$weekstart", "input")

    exd = JSON.parsefile(joinpath(sti_dataset1, "exogenprices_prognose1.json"))
    exogen = JulES.getelements(exd, sti_dataset1)

    add = JSON.parsefile(joinpath(sti_dataset, "aggdetd2.json"))
    aggdetd = JulES.getelements(add, sti_dataset)

    ipad = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoseragg$weatheryear.json"))
    agginflow = JulES.getelements(ipad, sti_dataset1)

    thd = JSON.parsefile(joinpath(sti_dataset, "termisk1.json"))
    thermal = JulES.getelements(thd, sti_dataset)

    wsd = JSON.parsefile(joinpath(sti_dataset, "vindsol.json"))
    windsol = JulES.getelements(wsd, sti_dataset)

    trd = JSON.parsefile(joinpath(sti_dataset1, "nett.json"))
    transm = JulES.getelements(trd)

    cod = JSON.parsefile(joinpath(sti_dataset, "forbruk5.json"))
    cons = JulES.getelements(cod, sti_dataset)

    fpd = JSON.parsefile(joinpath(sti_dataset1, "brenselspriser.json"))
    fuel = JulES.getelements(fpd, sti_dataset1)

    nud = JSON.parsefile(joinpath(sti_dataset1, "nuclear.json"))
    nuclear = JulES.getelements(nud, sti_dataset1)

    dse = JSON.parsefile(joinpath(sti_dataset, "tidsserier_detd.json"))
    detdseries = JulES.getelements(dse, sti_dataset)

    dda = JSON.parsefile(joinpath(sti_dataset, "dataset_detd.json"))
    detdstructure = JulES.getelements(dda)

    ipd = JSON.parsefile(joinpath(sti_dataset1, "tilsigsprognoser$weatheryear.json"))
    inflow = JulES.getelements(ipd, sti_dataset1)

    progelements = vcat(exogen, aggdetd, thermal, windsol, transm, cons, agginflow, fuel, nuclear)
    aggstartmagdict = JSON.parsefile(joinpath(sti_dataset1, "aggstartmagdict.json"), dicttype=Dict{String, Float64})

    if JulES.get_onlyagghydro(settings)
        startmagdict = Dict()
        detailedrescopl = Dict()
        return Dict("elements" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl)
    else
        elements = vcat(exogen, detdseries, detdstructure, thermal, windsol, transm, cons, inflow, fuel, nuclear)
        startmagdict = JSON.parsefile(joinpath(sti_dataset1, "startmagdict.json"))
        detailedrescopl = JSON.parsefile(joinpath(sti_dataset, "magasin_elspot.json"))
        return Dict("elements" => elements, "elements_ppp" => progelements, "startmagdict" => startmagdict, "aggstartmagdict" => aggstartmagdict, "detailedrescopl" => detailedrescopl)
    end
end



getdataset (generic function with 1 method)

### Run JulES and keep the results

In [28]:

dataset = getdataset(config, weatheryear)
input = JulES.DefaultJulESInput(config, dataset, datayear, weatheryear)
JulES.cleanup_jules(input)
data = JulES.run_serial(input)

Time parameters
  0.000046 seconds (56 allocations: 2.250 KiB)
Handle elements
  0.000959 seconds (14.37 k allocations: 716.344 KiB)
Add local dbs
  0.011866 seconds (721 allocations: 33.375 KiB)
Add local cores
  0.011610 seconds (661 allocations: 25.875 KiB)
Add local input
  2.818573 seconds (368.17 k allocations: 12.482 MiB)
Add local dummyobjects
  0.474567 seconds (1.13 M allocations: 89.280 MiB)
Add local subsystems
Number of shortterm storagesystems 0
Number of longterm storagesystems 19
  0.120413 seconds (1.29 M allocations: 38.654 MiB)
Add local scenmod
  0.012252 seconds (770 allocations: 41.453 KiB)
Add local problem distribution
[(1, 1), (2, 2), (3, 3), (4, 4)]
[(1, 1, 1), (2, 1, 2), (3, 1, 3), (4, 1, 4), (1, 2, 1), (2, 2, 2), (3, 2, 3), (4, 2, 4), (1, 3, 1), (2, 3, 2), (3, 3, 3), (4, 3, 4), (1, 4, 1), (2, 4, 2), (3, 4, 3), (4, 4, 4), (1, 5, 1), (2, 5, 2), (3, 5, 3), (4, 5, 4), (1, 6, 1), (2, 6, 2), (3, 6, 3), (4, 6, 4), (1, 7, 1), (2, 7, 2), (3, 7, 3), (4, 7, 4), (1, 8, 

Row,model,update,solve,other,total
,String,Float64,Float64,Float64,Float64
1,long,0.0622057,0.0624844,0.000145875,0.124836
2,med,0.159805,0.0462497,0.00024815,0.206303
3,short,0.0213106,0.0637418,0.00212206,0.0871744
4,evp,0.000564939,0.00111545,0.000409121,0.00208951
5,mp,0.000525353,0.000408376,0.00637447,0.0073082
6,sp,0.00392161,0.00146134,0.0175914,0.0229744
7,clearing,0.00726404,0.0163591,0.00297384,0.026597


Row,core,tot,evp_tot,mp_tot,sp_tot,evp_u,evp_s,evp_o,mp_u,mp_s,mp_fin,mp_o,sp_u,sp_s,sp_o
,Any,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,5.0,missing,missing,0.00366885,0.047041,missing,missing,missing,0.00016065,0.00104145,1.07e-5,0.00245605,0.001953,0.0032895,0.0417985
2,6.0,missing,missing,0.00784925,0.0445661,missing,missing,missing,0.00433955,0.0008883,9.7e-6,0.0026117,0.0164144,0.0026951,0.0254566
3,7.0,missing,missing,0.0039487,0.0476137,missing,missing,missing,0.00016605,0.0009603,1.285e-5,0.0028095,0.00181255,0.00435895,0.0414422
4,8.0,missing,missing,0.0043277,0.0468604,missing,missing,missing,0.0001375,0.00067845,1.19e-5,0.00349985,0.00159385,0.00298875,0.0422778
5,9.0,missing,missing,0.00864105,0.0524199,missing,missing,missing,0.0044209,0.00108495,1.05e-5,0.0031247,0.0172364,0.00380175,0.0313817
6,10.0,missing,missing,0.0024994,0.0048249,missing,missing,missing,8.01e-5,0.00036325,4.25e-6,0.0020518,0.00073685,0.00082705,0.003261
7,1.0,0.0936067,0.0105024,0.00166375,0.0814405,0.0027077,0.0053214,0.000309169,0.0001478,0.0006607,1.685e-5,0.0008384,0.0146342,0.002408,0.0643983
8,2.0,0.0935497,0.0092967,0.035434,0.048819,0.0026195,0.0049744,0.00021285,0.0001623,0.000708,1.27e-5,0.034551,0.0166602,0.00318705,0.0289718
9,3.0,0.0900464,0.0101539,0.0351455,0.044747,0.00273285,0.00561645,0.000225575,0.00020325,0.00067895,1.045e-5,0.0342529,0.001761,0.00180095,0.0411851


Row,subix,tot,evp_tot,mp_tot,sp_tot,evp_u,evp_s,evp_o,mp_u,mp_s,mp_fin,mp_o,sp_u,sp_s,sp_o
,Any,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Any,Any,Any,Any,Float64?,Float64?,Float64?
1,9.0,0.0526601,0.00216395,0.0073893,0.0431069,0.0005453,0.00121925,4.9925e-5,0.00435645,0.00073285,6.75e-6,0.00229325,0.0164583,0.00219295,0.0244556
2,16.0,0.0464831,0.0020718,0.00612855,0.0382827,0.00052985,0.0011684,4.66937e-5,0.00425605,0.0005668,4.35e-6,0.00130135,0.015662,0.0018578,0.020763
3,1.0,0.0458836,0.0034409,0.00066885,0.0417739,0.0008484,0.0017755,0.000102125,7.105e-5,0.00034865,9.6e-6,0.00023955,0.0136379,0.0016086,0.0265274
4,2.0,0.0455945,0.00262075,0.0336877,0.00928605,0.0006242,0.0013853,7.64063e-5,0.00010035,0.0003624,6.0e-6,0.033219,0.00085,0.0015899,0.00684615
5,18.0,0.0445089,0.0023109,0.001761,0.040437,0.0005443,0.0013721,4.93125e-5,6.215e-5,0.00033,4.95e-6,0.0013639,0.00081635,0.0015164,0.0381043
6,12.0,0.0434271,0.0021478,0.00174625,0.039533,0.0005397,0.00119385,5.17812e-5,6.195e-5,0.0003456,6.7e-6,0.001332,0.0158102,0.00159715,0.0221257
7,17.0,0.0431038,0.002272,0.0016643,0.0391675,0.00054115,0.0013658,4.56312e-5,8.25e-5,0.00040005,4.9e-6,0.00117685,0.0009384,0.0015987,0.0366304
8,13.0,0.0426811,0.00171015,0.0015548,0.0394161,0.00054955,0.00083785,4.03438e-5,0.0001024,0.00031755,4.25e-6,0.0011306,0.00090095,0.0007996,0.0377156
9,11.0,0.042231,0.0015696,0.0009949,0.0396666,0.0005419,0.000699,4.10875e-5,7.675e-5,0.00031205,7.25e-6,0.00059885,0.0009962,0.0007994,0.037871


Dict{Any, Any} with 23 entries:
  "demandbalancenames" => SubString{String}["SORLAND", "SORLAND", "SORLAND", "S…
  "endvaluetimes"      => Dict{Any, Any}((1, 12)=>[0.000158 0.0004999 0.00078; …
  "resindex"           => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-3…
  "areanames"          => ["SORLAND"]
  "priceindex"         => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-2…
  "batmatrix"          => Matrix{Float64}(undef, 128, 0)
  "stateindex"         => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-3…
  "statematrix"        => [547.673 4078.56 … 0.174767 0.0; 547.673 4078.56 … 0.…
  "statenames"         => ["Reservoir_FINNMARK_hydro_reservoir_max", "Reservoir…
  "resnames"           => ["Reservoir_SORLAND_hydro_reservoir"]
  "demandvalues"       => [0.0 0.0 … 0.0831064 1.40595; 0.0 0.94242 … 0.0476586…
  "batindex"           => [DateTime("2024-01-29T00:00:00"), DateTime("2024-01-3…
  "resmatrix"          => [14.4029; 14.3558; … ; 13.7965; 13.7811;;]
  "supplyba

### Code to show results
- We don't show any results for this demo

In [ ]:
powerbalancenames = data["areanames"]
prices = data["pricematrix"]
x1 = data["priceindex"]

hydronames = data["resnames"]
hydrolevels = data["resmatrix"]
x2 = data["resindex"]

batterynames = data["batnames"]
batterylevels = data["batmatrix"]
x2 = data["batindex"]

statenames = data["statenames"]
statematrix = data["statematrix"]
x3 = data["stateindex"]

supplyvalues = data["supplyvalues"]
supplynames = data["supplynames"]
supplybalancenames = data["supplybalancenames"]

demandvalues = data["demandvalues"]
demandnames = data["demandnames"]
demandbalancenames = data["demandbalancenames"];

# storagevalues = data["storagevalues"]
# storagenames = data["storagenames"]
# scenarionames = data["scenarionames"]
# shorts = data["shorts"]
# skipfactor = data["skipfactor"];

In [ ]:
# a = 0.5
# b = -4
# c = 10
# # a = 1
# # b = 0
# # c = 5
# # a = 0
# # b = 0
# # c = 1
# numscen = 7
# x = collect(-numscen+1:2:numscen-1)
# y = a .* x .^ 2 .+ x .* b .+ c
# display(y/sum(y)) # show chosen weights

In [ ]:
# Plot prices
idxwohub = findall(x -> !occursin("HUB", x), powerbalancenames) # remove hubs, not active in 2025 dataset
display(plot(x1, prices[:,idxwohub]*100, labels=reshape(powerbalancenames[idxwohub],1,length(powerbalancenames[idxwohub])), size=(800,500), title="Prices", ylabel="€/MWh"))

# # Plot supplies and demands
# maxdemsup = isempty(supplyvalues) ? maximum(demandvalues) : (isempty(demandvalues) ? maximum(supplyvalues) : max(maximum(demandvalues), maximum(supplyvalues)))
# supplychart = plot(x1, supplyvalues,labels=reshape(supplynames,1,length(supplynames)),title="Supply", ylabel = "GWh/h", ylims=(0,maxdemsup))
# demandchart = plot(x1, demandvalues,labels=reshape(demandnames,1,length(demandnames)),title="Demand", ylabel = "GWh/h", ylims=(0,maxdemsup))
sumsupplyvalues = sum(supplyvalues,dims=2)
sumdemandvalues = sum(demandvalues,dims=2)
maxdemsup = isempty(sumsupplyvalues) ? maximum(sumdemandvalues) : (isempty(sumdemandvalues) ? maximum(sumsupplyvalues) : max(maximum(sumdemandvalues), maximum(sumsupplyvalues)))
supplychart = areaplot(x1,sumsupplyvalues,title="Supply", ylabel = "GWh/h")
demandchart = areaplot(x1,sumdemandvalues,title="Demand", ylabel = "GWh/h")
display(plot([supplychart,demandchart]...,layout=(1,2),size=(800,500)))
# display(plot(supplychart,size=(800,500)))

# Plot storages
# display(areaplot(x2, hydrolevels1,labels=reshape(hydronames,1,length(hydronames)),size=(800,500),title="Reservoir levels", ylabel = "TWh")) #
display(areaplot(x2, dropdims(sum(hydrolevels,dims=2),dims=2),labels="Total",size=(800,500),title="Reservoir levels", ylabel = "TWh")) #

# display(areaplot(x1, dropdims(sum(batterylevels,dims=2),dims=2),labels="Total",size=(800,500),title="Short term storage levels", ylabel = "GWh")) #

# Plot list of yearly mean production and demand for each supply/demand TODO: split demand/supply and transmission
meandemand = dropdims(mean(demandvalues,dims=1),dims=1)
meanproduction = dropdims(mean(supplyvalues,dims=1),dims=1)
supplydf = sort(DataFrame(Supplyname = supplynames, Yearly_supply_TWh = meanproduction*8.76),[:Yearly_supply_TWh], rev = true)
demanddf = sort(DataFrame(Demandname = demandnames, Yearly_demand_TWh = meandemand*8.76),[:Yearly_demand_TWh], rev = true)
supplydf[!,:ID] = collect(1:length(supplynames))
demanddf[!,:ID] = collect(1:length(demandnames))
joineddf = select!(outerjoin(supplydf,demanddf;on=:ID),Not(:ID))
pretty_table(joineddf, show_subheader = true)

# Check that total supply equals total demand
pretty_table(combine(joineddf, [:Yearly_supply_TWh, :Yearly_demand_TWh] .=> sum∘skipmissing), show_subheader = true)

# Plot list of yearly income and cost for each supply/demand (only works if exogenprices are collected)
supplyrev = copy(supplyvalues)
for (i,supplybalancename) in enumerate(supplybalancenames)
    idx = findfirst(isequal(supplybalancename), powerbalancenames)
    supplyrev[:,i] .= supplyrev[:,i] .* prices[:,idx]
end
demandrev = copy(demandvalues)
for (i,demandbalancename) in enumerate(demandbalancenames)
    idx = findfirst(isequal(demandbalancename), powerbalancenames)
    demandrev[:,i] .= demandrev[:,i] .* prices[:,idx]
end
meandemandrev = dropdims(mean(demandrev,dims=1),dims=1)
meanproductionrev = dropdims(mean(supplyrev,dims=1),dims=1)
supplyrevdf = sort(DataFrame(Supplyname = supplynames, Yearly_rev_mill€ = meanproductionrev*8.76),[:Yearly_rev_mill€], rev = true)
demandrevdf = sort(DataFrame(Demandname = demandnames, Yearly_cost_mill€ = meandemandrev*8.76),[:Yearly_cost_mill€], rev = true)
supplyrevdf[!,:ID] = collect(1:length(supplynames))
demandrevdf[!,:ID] = collect(1:length(demandnames))
joinedrevdf = select!(outerjoin(supplyrevdf,demandrevdf;on=:ID),Not(:ID))
# pretty_table(joinedrevdf, show_subheader = true)

# Sum revenues and cost
pretty_table(combine(joinedrevdf, [:Yearly_rev_mill€, :Yearly_cost_mill€] .=> sum∘skipmissing), show_subheader = true)

# # Plot storagevalues for each reservoir and scenarios
# maxlongtermstorages = 40
# maxshorttermstorages = 40
# shortindex = x3
# medindex = x3[1:Int(skipfactor):Int(end-skipfactor)]
# numop = length(findall(sn -> occursin("Operative", sn), scenarionames))
# numscen = length(scenarionames) - numop
# j = 0
# k = 0
# for (i, storagename) in enumerate(storagenames)
#     if shorts[i]
#         j += 1
#         j > maxshorttermstorages && continue
#         storagevalues_ = storagevalues[:,:,:]
#         index = shortindex
#     else
#         k += 1
#         k > maxlongtermstorages && continue
#         storagevalues_ = storagevalues[1:Int(skipfactor):Int(end-skipfactor),:,:]
#         index = medindex
#     end
#     p = plot(index, storagevalues_[:,1:numscen,i] * -100, size=(800,500), title="Storagevalues scenario and operative for " * storagename, labels=reshape(scenarionames[1:numscen], 1, numscen), ylabel="€/MWh")
#     plot!(p, index, storagevalues_[:,numscen+1:end,i] * -100, labels=reshape(scenarionames[numscen+1:end], 1, numop), linewidth=5)
#     display(p)
# end